In [1]:
import sys
sys.path.append(r"../")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import pingouin as pg
#from main import create_dataset
import dtale
import os
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
from statsmodels.stats.contingency_tables import mcnemar


In [2]:
times = {'intake': ['intake_arm_1', 'pre_treatment_arm_1'],
         'time2' : ['5th_session_arm_1', 'control_5weeks_arm_1'],
         'time3' : ['followup_3month_arm_1', 'control_3month_arm_1', 'control_6month_arm_1'],
        }
# if not os.path.exists("Liat graphs/plots"):
#     os.mkdir("Liat graphs/plots")

group_names = {
    '0': 'invalid',
    '1': 'ipt', # חירום
    '2' : 'tau', # רגיל
    '3': 'control' # מינימלית
}

# for time in times.keys():
# #     print(time)
#     create_dataset(event_names = times[time], path = f"Liat graphs/data/{time}/DeppClinic_patient_data1.csv")


In [3]:
df_times = {
    'intake': pd.read_csv(r'data/intake_processed.csv'),
    'time2' : pd.read_csv(r'data/time2_processed.csv'),
    'time3' : pd.read_csv(r'data/time3_processed.csv')
         }

intake_target_variables =  ['suicidal_ideation', 'suicidal_behavior',
        'suicidal_attempt', 'ER', 'NSSI']

time2_target_variables = ['suicidal_ideation', 'suicidal_behavior', 
        'suicidal_attempt', 'ER', 'Psychiatric', 
        'NSSI', 'finished_treatment']

target_variables = {
    'intake': intake_target_variables,
    'time2': time2_target_variables, 
    'time3': time2_target_variables}
        

In [4]:
for time in times:
    df = df_times[time]
    df_times[time] = df[df['age_child_pre'] < 12.5]

In [5]:
def rename_groups(df, group_names):
    df['group'] = '0'
    df.loc[df['group___1'] == 1, 'group'] = '1'
    df.loc[df['group___2'] == 1, 'group'] = '2'
    df.loc[df['group___3'] == 1, 'group'] = '3'
    df['group'] = df['group'].map(group_names)
    return df

for time in times:
    df = df_times[time]
    df_times[time] = rename_groups(df, group_names)
    df_times[time] = df_times[time][df_times[time]["group"].isin(["ipt", "control"])]

In [7]:
df_times['intake'].id.to_csv("old_process_intake_ids.csv", index=False)

df_times['time2'].id.to_csv("old_process_time2_ids.csv", index=False)

df_times['time3'].id.to_csv("old_process_time3_ids.csv", index=False)

In [8]:
df_times['time3'].groupby('group')['suicidal_ideation'].value_counts()

group    suicidal_ideation
control  0                    15
         1                     8
ipt      0                    22
         1                     8
Name: count, dtype: int64

In [23]:
suicidal_ideation_life_intake = [
 #   'c_ssrs_t_life_1_clin',
    'c_ssrs_t_life_2_clin',
    'c_ssrs_t_life_3_clin',
    'c_ssrs_t_life_4_clin',
    'c_ssrs_t_life_5_clin',

#    'c_ssrs_t_2weeks_1_clin',
    'c_ssrs_t_2weeks_2_clin',
    'c_ssrs_t_2weeks_3_clin',
    'c_ssrs_t_2weeks_4_clin',
    'c_ssrs_t_2weeks_5_clin',

    #'c_ssrs_1_life',
    'c_ssrs_2_life',
    'c_ssrs_3_life',
    'c_ssrs_4_life',
    'c_ssrs_5_life',

    #'c_ssrs_1_2weeks',
    'c_ssrs_2_2weeks',
    'c_ssrs_3_2weeks',
    'c_ssrs_4_2weeks',
    'c_ssrs_5_2weeks',
  #  'chameleon_ideation_stu'

]


In [9]:
suicidal_ideation_time2 = [

    'c_ssrs_t_last_1_clin',
    'c_ssrs_t_last_2_clin',
    'c_ssrs_t_last_3_clin',
    'c_ssrs_t_last_4_clin',
    'c_ssrs_t_last_5_clin',

 #   'c_ssrs_t_2weeks_1_clin',
    'c_ssrs_t_2weeks_2_clin',
    'c_ssrs_t_2weeks_3_clin',
    'c_ssrs_t_2weeks_4_clin',
    'c_ssrs_t_2weeks_5_clin',

    'c_ssrs_1',
    'c_ssrs_2',
    'c_ssrs_3',
    'c_ssrs_4',
    'c_ssrs_5',

    #'chameleon_ideation_stu'
]

In [10]:
df_times['time2'].groupby('group')['suicidal_ideation'].value_counts()

group    suicidal_ideation
control  0                    14
         1                    14
ipt      0                    19
         1                    13
Name: count, dtype: int64

In [11]:
df_times['intake'].groupby('group')['suicidal_ideation'].value_counts()

group    suicidal_ideation
control  0                    19
         1                    17
ipt      1                    22
         0                    14
Name: count, dtype: int64

In [14]:
in_id = ['E5541', 'H0740', 'S5556', 'Y7918', 'D7663', 'T7989', 'I0658',
       'H1354', 'R5907', 'T0543', 'I6535', 'N4125', 'S6442', 'E3613',
       'T5527', 'I4694', 'R7617', 'N6117', 'A1651', 'A3781', 'T4075']

In [24]:
d = df_times['intake'].query("group == 'control' & suicidal_ideation == 0")


d[d.id.isin(in_id)]

,id,gender,redcap_event_name,age_child_pre,parents_born_m,parents_born_2_m,born_child_m,born_child_2_m,parent_religion_m,parent_religion_other_m,...,group___2,group___3,suicidal_ideation,suicidal_behavior,suicidal_attempt,ER,Psychiatric,NSSI,finished_treatment,group
29,Y7918,1.0,intake_arm_1,11.0,1.0,NaN,1.0,NaN,1.0,NaN,...,0.0,1.0,0,0,0,0,0,0,0,control
45,T7989,2.0,intake_arm_1,11.0,2.0,אורוגואי,1.0,NaN,1.0,NaN,...,0.0,1.0,0,0,0,0,0,0,0,control
110,R5907,NaN,intake_arm_1,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0,0,0,0,0,0,0,control
220,I6535,1.0,intake_arm_1,9.0,1.0,NaN,1.0,NaN,NaN,NaN,...,0.0,1.0,0,0,0,0,0,0,0,control
287,E3613,1.0,intake_arm_1,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,0,0,0,0,0,0,0,control
387,A3781,2.0,intake_arm_1,11.0,1.0,NaN,1.0,NaN,2.0,NaN,...,0.0,1.0,0,0,0,0,0,0,0,control


In [25]:
df_times['intake'].query("id == 'T7989'")[suicidal_ideation_life_intake + ['suicidal_ideation', 'id', 'group']]

,c_ssrs_t_life_2_clin,c_ssrs_t_life_3_clin,c_ssrs_t_life_4_clin,c_ssrs_t_life_5_clin,c_ssrs_t_2weeks_2_clin,c_ssrs_t_2weeks_3_clin,c_ssrs_t_2weeks_4_clin,c_ssrs_t_2weeks_5_clin,c_ssrs_2_life,c_ssrs_3_life,c_ssrs_4_life,c_ssrs_5_life,c_ssrs_2_2weeks,c_ssrs_3_2weeks,c_ssrs_4_2weeks,c_ssrs_5_2weeks,suicidal_ideation,id,group
45,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0,T7989,control


In [12]:
for target in target_variables['intake']:
    intake = df_times['intake']
    time2 = df_times['time2']
    
    intake = intake[['group', target]]
    time2 = time2[['group', target]]

    intake_values = intake.groupby('group')[target].sum()[['ipt', 'control']].values
    intake_gb = intake.groupby('group')[target].sum()[['ipt', 'control']]

    time2_values = time2.groupby('group')[target].sum()[['ipt', 'control']].values
    time2_gb = time2.groupby('group')[target].sum()[['ipt', 'control']]

    df = pd.DataFrame({'intake': intake_values, 'time2': time2_values})
    print(f"\n\n{target =}\n\n{intake_gb = }\n\n{time2_gb = }\n\n")
#     print('\n\n\n\n\n',target)
#     print(mcnemar(df.values, exact=False))



target ='suicidal_ideation'

intake_gb = group
ipt        22
control    17
Name: suicidal_ideation, dtype: int64

time2_gb = group
ipt        13
control    14
Name: suicidal_ideation, dtype: int64




target ='suicidal_behavior'

intake_gb = group
ipt        4
control    3
Name: suicidal_behavior, dtype: int64

time2_gb = group
ipt        7
control    5
Name: suicidal_behavior, dtype: int64




target ='suicidal_attempt'

intake_gb = group
ipt        4
control    2
Name: suicidal_attempt, dtype: int64

time2_gb = group
ipt        2
control    1
Name: suicidal_attempt, dtype: int64




target ='ER'

intake_gb = group
ipt        1
control    0
Name: ER, dtype: int64

time2_gb = group
ipt        0
control    0
Name: ER, dtype: int64




target ='NSSI'

intake_gb = group
ipt        4
control    6
Name: NSSI, dtype: int64

time2_gb = group
ipt        5
control    5
Name: NSSI, dtype: int64




In [9]:
for target in target_variables['intake']:
    intake = df_times['intake']
    time3 = df_times['time3']
    
    intake = intake[['group', target]]
    time3 = time3[['group', target]]

    intake_values = intake.groupby('group')[target].sum()[['ipt', 'control']].values
    time3_values = time3.groupby('group')[target].sum()[['ipt', 'control']].values

    df = pd.DataFrame({'intake': intake_values, 'time3': time3_values})
    print('\n\n\n\n\n', target)
    print(mcnemar(df.values, exact=False))






 suicidal_ideation
pvalue      1.0
statistic   0.0





 suicidal_behavior
pvalue      0.16913144470266755
statistic   1.890625





 suicidal_attempt
pvalue      0.16913144470266755
statistic   1.890625





 ER
pvalue      0.10017829422626791
statistic   2.7027027027027026





 NSSI
pvalue      0.16913144470266755
statistic   1.890625
